In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/KavehSam/VPSNN.git vpsnn-repo
%cd vpsnn-repo
!ls

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join("."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import os.path
import random

import get_params as params
import torch
import torch.nn as nn
import torch.optim as optim
from classifier_models.utils import return_model_accuracy
from classifier_models.vpnn import (FCMVPCSNN, FCMVPSNN, FCTMVPSNN, FCVPNN,
                                    FCVPSNN, FCVPTSNN)
from dataloaders.load_dataset import load_ecg_real
from nn_layers.vp_layers import *


In [ ]:
%cd notebooks
params.get('network_configs/ECG_VPNN.yaml')

torch.manual_seed(params.SEED_NUM)
random.seed(params.SEED_NUM)

data_path = os.path.join('../data', 'ecg', 'mitbih')
train_loader, test_loader = load_ecg_real(params.batch_size, data_path)
number_of_observations = len(train_loader.dataset)


In [ ]:
if params.classifier_name == 'FCVPNN':
    '''FCVPNN - setup'''
    classifier = FCVPNN(params.vp_latent_dim, params.num_classes,
                        device=params.device, penalty=params.L2penalty, init_vp=params.init_vp)
elif params.classifier_name == 'FCVPSNN':
    '''FCVPSNN - setup'''
    init_vp = [0.1, 0]
    classifier = FCVPSNN(params.vp_latent_dim, params.num_classes, device=params.device,
                         penalty=params.L2penalty, init_vp=init_vp, n_steps=params.n_steps)
elif params.classifier_name == 'FCVPTSNN':
    '''FCVPTSNN - setup'''
    init_vp = [0.001, 0.001]
    classifier = FCVPTSNN(params.vp_latent_dim, params.num_classes, device=params.device,
                          penalty=params.L2penalty, init_vp=init_vp, n_steps=params.n_steps)
elif params.classifier_name == 'FCMVPSNN':
    '''FCMVPSNN - setup'''
    init_mvp = [random.random() for i in range(2 * params.n_steps)]
    classifier = FCMVPSNN(params.vp_latent_dim, params.num_classes, device=params.device,
                          penalty=params.L2penalty, init_vp=init_mvp, n_steps=params.n_steps, m_vp=params.n_steps)
elif params.classifier_name == 'FCTMVPSNN':
    '''FCTMVPSNN - setup'''
    init_mvp = [random.random()/2 for i in range(2 * params.n_steps)]
    classifier = FCTMVPSNN(params.vp_latent_dim, params.num_classes, device=params.device,
                           penalty=params.L2penalty, init_vp=init_mvp, n_steps=params.n_steps, m_vp=params.n_steps)
elif params.classifier_name == 'FCMVPCSNN':
    '''FCMVPCSNN - setup'''
    init_mvp = [random.random() for i in range(2 * params.n_steps)]
    # insert some weights which prooved to be suitable for ECG classification
    init_mvp[0:2] = [0.1, 0.0]
    classifier = FCMVPCSNN(params.vp_latent_dim, params.num_classes, device=params.device,
                           penalty=params.L2penalty, init_vp=init_mvp, n_steps=params.n_steps, m_vp=params.n_steps)

classifier.to(params.device)


In [ ]:
# define objective and optimizer functions
criterion = nn.BCELoss()
optimizer = optim.Adagrad(classifier.parameters(), 
                       lr=params.lr,
                       lr_decay=params.lr_decay,
                       weight_decay=0.001)


In [ ]:
# loop over the dataset multiple times
for epoch in range(params.epochs_cls):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(params.device), data[1].to(params.device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    # print epoch performance
    with torch.no_grad():
        train_acc = return_model_accuracy(classifier, train_loader, params.device)
        test_acc = return_model_accuracy(classifier, test_loader, params.device)
        print(f'[Epoch: {epoch + 1}], Train accuracy: {train_acc:.2f}%, Test accuracy: {test_acc:.2f}%')
        print(f'[Epoch: {epoch + 1}], Train accuracy: {train_acc:.2f}%, Test accuracy: {test_acc:.2f}%', file=open('output.txt', 'a'))

print('Finished Training')